In [1]:
from random import sample
import random
from tqdm import tqdm
import pandas as pd
import requests
import time
import DHB as dhb
import my_utils as mu
tqdm.pandas()

In [14]:
dhb.db_open()
df = dhb.sql("select * from match_aram")
dhb.db_close()

OPEN
close


# . 신발 픽률

In [15]:
df["WIN"] = df.apply(lambda x: 1 if x["WIN"] == "True" else 0 , axis=1)
champ_lst = list(df.groupby("CHAMPIONNAME").count().reset_index()["CHAMPIONNAME"])
url = "http://ddragon.leagueoflegends.com/cdn/11.24.1/data/ko_KR/item.json"
res = requests.get(url).json()
item_key = list(res['data'].keys())
itemurl = list(map(lambda x: "http://ddragon.leagueoflegends.com/cdn/11.24.1/img/item/"+x+".png", item_key))
item_lst = dict(zip(item_key,itemurl))

In [79]:
def boots_img(row):
    if row.CHAMPIONNAME != "Cassiopeia":
        tmp = []
        try:
            tmp += list(map(lambda x: item_lst[x], row["F_BOOTS"].split("|")))
        except:
            tmp += ["NoShoes"]
            
        tmp += [str(row.F_PICK_RATE)]
        tmp += [str(row.F_WIN_RATE)]
        
        tmp += list(map(lambda x: item_lst[x], row["S_BOOTS"].split("|")))
        tmp += [str(row.S_PICK_RATE)]
        tmp += [str(row.S_WIN_RATE)]
       
        tmp = "|".join(tmp)
    else:
        tmp = "0"
    return tmp

In [87]:
def boots_build(df):   
    champ_games = df.groupby("CHAMPIONNAME")[["GAMEID"]].count().rename(columns = {"GAMEID":"GAMES"}).reset_index()
    boots_games = df.groupby(["CHAMPIONNAME","BOOTS"])[["WIN"]].count().rename(columns={"WIN":"BOOTS_GAMES"}).reset_index()
    boots_win = df.groupby(["CHAMPIONNAME","BOOTS"])[["WIN"]].sum().reset_index()
    boots_win_pick = pd.merge(pd.merge(boots_games, boots_win, on=["CHAMPIONNAME","BOOTS"]),champ_games, on="CHAMPIONNAME")
    boots_win_pick["PICK_RATE"] = round(boots_win_pick.BOOTS_GAMES / boots_win_pick.GAMES * 100,1)
    boots_win_pick["WIN_RATE"] = round(boots_win_pick.WIN / boots_win_pick.BOOTS_GAMES * 100,1)
    boots_win_pick = boots_win_pick.sort_values("PICK_RATE", ascending=False)

    tmp = []
    for i in champ_lst:
        if len(boots_win_pick[boots_win_pick.CHAMPIONNAME == i]) > 1:
            tmp_lst = []
            tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[0]["CHAMPIONNAME"])
            tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[0]["BOOTS"])
            tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[0]["PICK_RATE"])
            tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[0]["WIN_RATE"])

            if boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[1]["BOOTS"] != "0":
                tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[1]["BOOTS"])
                tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[1]["PICK_RATE"])
                tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[1]["WIN_RATE"])
            else:
                tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[2]["BOOTS"])
                tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[2]["PICK_RATE"])
                tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[2]["WIN_RATE"])

            tmp.append(tmp_lst)
        else:
            tmp_lst = []
            tmp_lst.append(boots_win_pick[boots_win_pick.CHAMPIONNAME == i].iloc[0]["CHAMPIONNAME"])
            tmp_lst.append("0")
            tmp_lst.append("0")
            tmp_lst.append("0")
            tmp_lst.append("0")
            tmp_lst.append("0")
            tmp_lst.append("0")
            tmp.append(tmp_lst)

    champ_boots = pd.DataFrame(tmp,columns=["CHAMPIONNAME","F_BOOTS","F_PICK_RATE","F_WIN_RATE","S_BOOTS","S_PICK_RATE","S_WIN_RATE"])

    champ_boots["BOOTS_IMG"] = champ_boots.apply(lambda x: boots_img(x), axis=1)
    
    return champ_boots

In [88]:
champ_boots = boots_build(df)

In [89]:
champ_boots

,CHAMPIONNAME,F_BOOTS,F_PICK_RATE,F_WIN_RATE,S_BOOTS,S_PICK_RATE,S_WIN_RATE,BOOTS_IMG
0,Aatrox,3111,53.0,50.8,3047,35.4,51.9,http://ddragon.leagueoflegends.com/cdn/11.24.1...
1,Ahri,3020,81.0,44.1,3158,11.5,48.1,http://ddragon.leagueoflegends.com/cdn/11.24.1...
2,Akali,3020,69.7,42.7,3111,22.5,43.3,http://ddragon.leagueoflegends.com/cdn/11.24.1...
3,Akshan,3006,57.2,43.5,3111,22.5,46.9,http://ddragon.leagueoflegends.com/cdn/11.24.1...
4,Alistar,3111,54.1,55.3,3047,35.0,53.4,http://ddragon.leagueoflegends.com/cdn/11.24.1...
...,...,...,...,...,...,...,...,...
152,Zed,3158,61.9,45.3,3111,26.0,45.0,http://ddragon.leagueoflegends.com/cdn/11.24.1...
153,Ziggs,3020,86.9,54.8,3158,7.7,50.0,http://ddragon.leagueoflegends.com/cdn/11.24.1...
154,Zilean,3158,66.9,47.4,3020,25.9,41.6,http://ddragon.leagueoflegends.com/cdn/11.24.1...
155,Zoe,3020,91.3,44.2,3158,4.4,37.9,http://ddragon.leagueoflegends.com/cdn/11.24.1...
